In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import simweights
import pickle
import os, sys
import re
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import pandas as pd
import tables
import h5py
import math
from scipy.stats import mstats
import matplotlib as mpl
import matplotlib.font_manager as font_manager


In [4]:
sys.path.append("/data/user/tvaneede/GlobalFit/reco_processing/notebooks/weighting")
from weights import *
from utils import *
from selections import selection_mask
from fonts import *
from plot_utils import *

In [5]:
# Append the custom module path
sys.path.append("/data/user/tvaneede/GlobalFit/reco_processing")

# Import the datasets module
from datasets import datasets

# set the inputs
reco_versions = ["evtgen_v1_rec_v2", "spice_tau_reco"]

# Dynamically select the desired dataset
simulation_datasets = {}
for reco_version in reco_versions: simulation_datasets[reco_version] = getattr(datasets, reco_version)

In [6]:
livetime_yr = 11.687
livetime_s  = livetime_yr * 365.25 * 24 * 3600 # 11.687 year

In [7]:
main_plotting_path = f"/data/user/tvaneede/GlobalFit/reco_processing/notebooks/compare_spice_ftp/output"
os.system(f"mkdir -p {main_plotting_path}")

0

In [8]:
# weight functions
spline_file = '/data/ana/Diffuse/NNMFit/MCEq_splines/v1.2.1/MCEq_splines_PRI-Gaisser-H4a_INT-SIBYLL23c_allfluxes.pickle'

# conventional            
flux_keys_conv =  ['conv_antinumu','conv_numu','conv_antinue','conv_nue','conv_antinutau','conv_nutau']
spline_object_conv = SplineHandler(spline_file, flux_keys_conv)
conv_flux = spline_object_conv.return_weight
generator_conv = lambda pdgid, energy, cos_zen: conv_flux(pdgid, energy, cos_zen)

# prompt
flux_keys_pr =  ['pr_antinumu','pr_numu','pr_antinue','pr_nue','pr_antinutau','pr_nutau']
spline_object_pr = SplineHandler(spline_file, flux_keys_pr)
pr_flux = spline_object_pr.return_weight
generator_pr = lambda pdgid, energy, cos_zen: pr_flux(pdgid, energy, cos_zen)

# astro
gamma_astro = 2.87
per_flavor_norm = 2.12
def AstroFluxModel(pdgid, energy, cos_zen):
    flux = 0.5*(per_flavor_norm*1e-18)*(energy/1e5)**-gamma_astro
    return flux

In [9]:
def open_datasets( simulation_dataset, keys_to_merge ):

    # open the files
    for key in simulation_dataset:
        print(f"----- Extracting files for {key}")
        simulation_dataset[key]['hdf_file'] = pd.HDFStore(simulation_dataset[key]['hdf_file_path'],'r')
        simulation_dataset[key]['weighter'] = simweights.NuGenWeighter( simulation_dataset[key]['hdf_file'] ,nfiles=simulation_dataset[key]['nfiles'])

    # merging files
    for new_key in keys_to_merge:
        print(f"----- Creating new key {new_key}")
        simulation_dataset[new_key] = {}
        simulation_dataset[new_key]['variables'] = {}
        simulation_dataset[new_key]['weighter'] = None

        for key in keys_to_merge[new_key]:
            
            print(f"Using {key}")
            # merge the weighters
            if simulation_dataset[new_key]['weighter'] == None:
                simulation_dataset[new_key]['weighter'] = simulation_dataset[key]['weighter']
            else: simulation_dataset[new_key]['weighter'] += simulation_dataset[key]['weighter']

    # calculate weights
    for key in simulation_dataset:
        simulation_dataset[key]['weights_astro'] = simulation_dataset[key]["weighter"].get_weights(AstroFluxModel) * livetime_s
        simulation_dataset[key]['weights_conv'] = simulation_dataset[key]["weighter"].get_weights(generator_conv) * livetime_s
        simulation_dataset[key]['weights_pr'] = simulation_dataset[key]["weighter"].get_weights(generator_pr) * livetime_s


    return simulation_dataset




In [10]:
keys_to_merge = {}

keys_to_merge["evtgen_v1_rec_v2"] = {
    "NuE" : ["NuE_midE", "NuE_highE"],
    "NuMu" : ["NuMu_midE", "NuMu_highE"],
    "NuTau" : ["NuTau_midE", "NuTau_highE"],
    "NuAll" : ['NuE', "NuMu", "NuTau"],
}

keys_to_merge["v2"] = {
    "NuE" : ["NuE_midE", "NuE_highE"],
    "NuMu" : ["NuMu_midE", "NuMu_highE"],
    "NuTau" : ["NuTau_midE", "NuTau_highE"],
    "NuAll" : ['NuE', "NuMu", "NuTau"],
}

keys_to_merge["spice_tau_reco"] = {
    "NuE" : ["NuE_midE1", "NuE_highE1", "NuE_midE2", "NuE_highE2"],
    "NuMu" : ["NuMu_midE1", "NuMu_highE1","NuMu_midE2", "NuMu_highE2"],
    "NuTau" : ["NuTau_midE1", "NuTau_highE1","NuTau_midE2", "NuTau_highE2"],
    "NuAll" : ['NuE', "NuMu", "NuTau"],

}


In [11]:
for key in simulation_datasets: simulation_datasets[key] = open_datasets( simulation_datasets[key], keys_to_merge[key] )

----- Extracting files for NuTau_midE
----- Extracting files for NuTau_highE
----- Extracting files for NuE_midE
----- Extracting files for NuE_highE
----- Extracting files for NuMu_midE
----- Extracting files for NuMu_highE
----- Creating new key NuE
Using NuE_midE
Using NuE_highE
----- Creating new key NuMu
Using NuMu_midE
Using NuMu_highE
----- Creating new key NuTau
Using NuTau_midE
Using NuTau_highE
----- Creating new key NuAll
Using NuE
Using NuMu
Using NuTau
----- Extracting files for NuTau_midE1
----- Extracting files for NuTau_highE1
----- Extracting files for NuTau_midE2
----- Extracting files for NuTau_highE2
----- Extracting files for NuE_midE1
----- Extracting files for NuE_highE1
----- Extracting files for NuE_midE2
----- Extracting files for NuE_highE2
----- Extracting files for NuMu_midE1
----- Extracting files for NuMu_highE1
----- Extracting files for NuMu_midE2
----- Extracting files for NuMu_highE2
----- Creating new key NuE
Using NuE_midE1
Using NuE_highE1
Using Nu

In [13]:
data = {}

for key in simulation_datasets:

    simulation_dataset = simulation_datasets[key]

    channel_data = {}

    for flavor in ['NuE', "NuMu", "NuTau"]:
        weights = simulation_dataset[flavor]["weighter"].get_weights(AstroFluxModel) * livetime_s
        rate = np.sum(weights)
        error = np.sqrt(np.sum(weights**2))
        channel_data[f"astro_{flavor}"] = f"{rate:.2f} ± {error:.2f}"

    # Conventional
    flavor = "NuAll"
    weights_conv = simulation_dataset[flavor]["weighter"].get_weights(generator_conv) * livetime_s
    rate_conv = np.sum(weights_conv)
    err_conv = np.sqrt(np.sum(weights_conv**2))
    channel_data["conv"] = f"{rate_conv:.2f} ± {err_conv:.2f}"

    # Prompt
    weights_prompt = simulation_dataset[flavor]["weighter"].get_weights(generator_pr) * livetime_s
    rate_prompt = np.sum(weights_prompt)
    err_prompt = np.sqrt(np.sum(weights_prompt**2))
    channel_data["prompt"] = f"{rate_prompt:.2f} ± {err_prompt:.2f}"

    data[key] = channel_data

# Create DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Optional: specify column order
columns_order = [f"astro_{flavor}" for flavor in ['NuE', 'NuMu', 'NuTau']] + ["conv", "prompt"]
df = df[columns_order]

# Display as string table
print(df.to_string())

                     astro_NuE    astro_NuMu   astro_NuTau          conv        prompt
evtgen_v1_rec_v2  56.20 ± 0.54  14.74 ± 0.22  33.91 ± 0.39  32.36 ± 0.97  12.47 ± 0.10
spice_tau_reco    56.77 ± 0.56  20.42 ± 0.22  34.89 ± 0.43  38.77 ± 0.95  13.41 ± 0.11


In [14]:
for key in simulation_datasets:

    data = {}
    simulation_dataset = simulation_datasets[key]

    channel_data = {}

    for flavor in keys_to_merge[key]["NuMu"]:
        weights = simulation_dataset[flavor]["weighter"].get_weights(AstroFluxModel) * livetime_s
        rate = np.sum(weights)
        error = np.sqrt(np.sum(weights**2))
        channel_data[f"astro_{flavor}"] = f"{rate:.2f} ± {error:.2f}"

    data[key] = channel_data

    # Create DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')

    # Display as string table
    print(df.to_string())

                 astro_NuMu_midE astro_NuMu_highE
evtgen_v1_rec_v2    13.92 ± 0.21      0.82 ± 0.04
               astro_NuMu_midE1 astro_NuMu_highE1 astro_NuMu_midE2 astro_NuMu_highE2
spice_tau_reco     19.28 ± 0.41       1.22 ± 0.04     19.18 ± 0.27       1.21 ± 0.03


I seem to have only 72-67%. Let's see if v2 of the reco is fine.

In [15]:
from datasets import datasets
import importlib

importlib.reload(datasets)

# set the inputs
reco_versions = ["v2", "spice_tau_reco"]

# Dynamically select the desired dataset
simulation_datasets = {}
for reco_version in reco_versions: simulation_datasets[reco_version] = getattr(datasets, reco_version)

In [16]:
for key in simulation_datasets: simulation_datasets[key] = open_datasets( simulation_datasets[key], keys_to_merge[key] )

----- Extracting files for NuTau_midE
----- Extracting files for NuTau_highE
----- Extracting files for NuE_midE
----- Extracting files for NuE_highE
----- Extracting files for NuMu_midE
----- Extracting files for NuMu_highE
----- Creating new key NuE
Using NuE_midE
Using NuE_highE
----- Creating new key NuMu
Using NuMu_midE
Using NuMu_highE
----- Creating new key NuTau
Using NuTau_midE
Using NuTau_highE
----- Creating new key NuAll
Using NuE
Using NuMu
Using NuTau
----- Extracting files for NuTau_midE1
----- Extracting files for NuTau_highE1
----- Extracting files for NuTau_midE2
----- Extracting files for NuTau_highE2
----- Extracting files for NuE_midE1
----- Extracting files for NuE_highE1
----- Extracting files for NuE_midE2
----- Extracting files for NuE_highE2
----- Extracting files for NuMu_midE1
----- Extracting files for NuMu_highE1
----- Extracting files for NuMu_midE2
----- Extracting files for NuMu_highE2
----- Creating new key NuE
Using NuE_midE1
Using NuE_highE1
Using Nu

In [17]:
for key in simulation_datasets:

    data = {}
    simulation_dataset = simulation_datasets[key]

    channel_data = {}

    for flavor in keys_to_merge[key]["NuMu"]:
        weights = simulation_dataset[flavor]["weighter"].get_weights(AstroFluxModel) * livetime_s
        rate = np.sum(weights)
        error = np.sqrt(np.sum(weights**2))
        channel_data[f"astro_{flavor}"] = f"{rate:.2f} ± {error:.2f}"

    data[key] = channel_data

    # Create DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')

    # Display as string table
    print(df.to_string())

   astro_NuMu_midE astro_NuMu_highE
v2    13.92 ± 0.21      0.82 ± 0.04
               astro_NuMu_midE1 astro_NuMu_highE1 astro_NuMu_midE2 astro_NuMu_highE2
spice_tau_reco     19.28 ± 0.41       1.22 ± 0.04     19.18 ± 0.27       1.21 ± 0.03


Also missing! Let's take a look at ftp_l3casc and do a cut myself

In [20]:
from datasets import datasets
import importlib

importlib.reload(datasets)

# set the inputs
reco_versions = ["ftp_l3casc", "spice_l3casc"]

# Dynamically select the desired dataset
simulation_datasets = {}
for reco_version in reco_versions: simulation_datasets[reco_version] = getattr(datasets, reco_version)

In [21]:
keys_to_merge["ftp_l3casc"] = {
    "NuE" : ["NuE_midE1", "NuE_highE1", "NuE_midE2", "NuE_highE2"],
    "NuMu" : ["NuMu_midE1", "NuMu_highE1","NuMu_midE2", "NuMu_highE2"],
    "NuTau" : ["NuTau_midE1", "NuTau_highE1","NuTau_midE2", "NuTau_highE2"],
    "NuAll" : ['NuE', "NuMu", "NuTau"],   
}

keys_to_merge["spice_l3casc"] = {
    "NuE" : ["NuE_midE1", "NuE_highE1", "NuE_midE2", "NuE_highE2"],
    "NuMu" : ["NuMu_midE1", "NuMu_highE1","NuMu_midE2", "NuMu_highE2"],
    "NuTau" : ["NuTau_midE1", "NuTau_highE1","NuTau_midE2", "NuTau_highE2"],
    "NuAll" : ['NuE', "NuMu", "NuTau"],
}

for key in simulation_datasets: simulation_datasets[key] = open_datasets( simulation_datasets[key], keys_to_merge[key] )

----- Extracting files for NuTau_midE1
----- Extracting files for NuTau_midE2
----- Extracting files for NuTau_highE1
----- Extracting files for NuTau_highE2
----- Extracting files for NuE_midE1
----- Extracting files for NuE_midE2
----- Extracting files for NuE_highE1
----- Extracting files for NuE_highE2
----- Extracting files for NuMu_midE1
----- Extracting files for NuMu_midE2
----- Extracting files for NuMu_highE1
----- Extracting files for NuMu_highE2
----- Creating new key NuE
Using NuE_midE1
Using NuE_highE1
Using NuE_midE2
Using NuE_highE2
----- Creating new key NuMu
Using NuMu_midE1
Using NuMu_highE1
Using NuMu_midE2
Using NuMu_highE2
----- Creating new key NuTau
Using NuTau_midE1
Using NuTau_highE1
Using NuTau_midE2
Using NuTau_highE2
----- Creating new key NuAll
Using NuE
Using NuMu
Using NuTau
----- Extracting files for NuTau_midE1
----- Extracting files for NuTau_highE1
----- Extracting files for NuTau_midE2
----- Extracting files for NuTau_highE2
----- Extracting files f

In [23]:
data = {}

key = "ftp_l3casc"

simulation_dataset = simulation_datasets[key]

channel_data = {}
channel_data_masked = {}

for flavor in keys_to_merge[key]["NuMu"]:
    weights = simulation_dataset[flavor]["weighter"].get_weights(AstroFluxModel) * livetime_s
    rate = np.sum(weights)
    error = np.sqrt(np.sum(weights**2))
    channel_data[f"astro_{flavor}"] = f"{rate:.2f} ± {error:.2f}"

    HESE_CausalQTot = simulation_dataset[flavor]["hdf_file"]["HESE_CausalQTot"].value
    mask = HESE_CausalQTot > 6000
    rate_masked = np.sum(weights[mask])
    error_masked = np.sqrt(np.sum(weights[mask]**2))
    channel_data_masked[f"astro_{flavor}"] = f"{rate_masked:.2f} ± {error_masked:.2f}"

data[key] = channel_data
data[f"{key}_masked"] = channel_data_masked

# Create DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Display as string table
print(df.to_string())

                  astro_NuMu_midE1 astro_NuMu_highE1 astro_NuMu_midE2 astro_NuMu_highE2
ftp_l3casc           890.37 ± 1.65       8.84 ± 0.05    894.52 ± 1.70       8.86 ± 0.06
ftp_l3casc_masked     13.94 ± 0.13       0.83 ± 0.01     14.18 ± 0.14       0.84 ± 0.02


Seems to be the same amount of events missing. Lets check one dataset of 0000000-0000999

In [24]:
file_path_NuMu_midE = "/data/user/tvaneede/GlobalFit/reco_processing/hdf/output/ftp_l3casc/NuMu_22645_0000000-0000999.h5"
hdf_NuMu_midE = pd.HDFStore(file_path_NuMu_midE,'r')
nfiles_NuMu_midE = 1000
weighter_NuMu_midE = simweights.NuGenWeighter( hdf_NuMu_midE, nfiles=nfiles_NuMu_midE)
weights_NuMu_midE = weighter_NuMu_midE.get_weights(AstroFluxModel) * livetime_s
rate_NuMu_midE = np.sum(weights_NuMu_midE)
mask_NuMu_midE = hdf_NuMu_midE["HESE_CausalQTot"].value > 6000
rate_masked_NuMu_midE = np.sum(weights_NuMu_midE[mask_NuMu_midE])

print("rate_NuMu_midE", rate_NuMu_midE, rate_masked_NuMu_midE)

file_path_NuMu_highE = "/data/user/tvaneede/GlobalFit/reco_processing/hdf/output/ftp_l3casc/NuMu_22644_0000000-0000999.h5"
hdf_NuMu_highE = pd.HDFStore(file_path_NuMu_highE,'r')
nfiles_NuMu_highE = 1000
weighter_NuMu_highE = simweights.NuGenWeighter( hdf_NuMu_highE, nfiles=nfiles_NuMu_highE)
weights_NuMu_highE = weighter_NuMu_highE.get_weights(AstroFluxModel) * livetime_s
rate_NuMu_highE = np.sum(weights_NuMu_highE)
mask_NuMu_highE = hdf_NuMu_highE["HESE_CausalQTot"].value > 6000
rate_masked_NuMu_highE = np.sum(weights_NuMu_highE[mask_NuMu_highE])

print("rate_NuMu_highE", rate_NuMu_highE, rate_masked_NuMu_highE)


rate_NuMu_midE 888.3088102238869 14.876935899724039
rate_NuMu_highE 8.933702850310775 0.9000591852292844


I am really starting to believe that we actually have fewer muon neutrinos at hese level for the ftp-v3 simulations. Lets make a hdf of the spice files at cascade level to see if there is a difference there as well?

Turns out one reco file was corrupted, so the hdf of that group of files was broken, see tools/find_error_in_log.py

Missing jobs: 0

Error jobs: 1
{'NuMu_22043_0000000-0000999': {'LOGDIR': '/scratch/tvaneede/reco/hdf_taupede_tianlu/spice_l3casc/hdf_dag_spice_l3casc/logs', 'JOBID': 'NuMu_22043_0000000-0000999', 'INPATH': '/data/sim/IceCube/2020/filtered/level3/cascade/neutrino-generator/22043/0000000-0000999', 'OUTFILE': '/data/user/tvaneede/GlobalFit/reco_processing/hdf/output/spice_l3casc/NuMu_22043_0000000-0000999.h5'}}

In [52]:
key = "spice_l3casc"
flavor = "NuMu_midE1"
weights = simulation_dataset[flavor]["weighter"].get_weights(AstroFluxModel) * livetime_s
HESE_CausalQTot = simulation_dataset[flavor]["weighter"].get_column("HESE_CausalQTot", "value")
HESE_CausalQTot_2 = simulation_dataset[flavor]["hdf_file"]["HESE_CausalQTot"]
I3MCWeightDict = simulation_dataset[flavor]["hdf_file"]["I3MCWeightDict"]

print(len(weights),len(I3MCWeightDict),len(HESE_CausalQTot), len(HESE_CausalQTot_2))
# mask = simulation_dataset[flavor]["weighter"].get_selection_mask()

# Extract identifiers from both tables
mc = simulation_dataset[flavor]["hdf_file"]["I3MCWeightDict"]
qtot = simulation_dataset[flavor]["hdf_file"]["HESE_CausalQTot"]

# Convert to DataFrames using common keys
id_keys = ["Run", "Event", "SubEvent"]

df_mc = pd.DataFrame({key: mc[key] for key in id_keys})
df_qtot = pd.DataFrame({key: qtot[key] for key in id_keys})

# Merge with indicator to find which are missing from qtot
merged = df_mc.merge(df_qtot, on=id_keys, how='left', indicator=True)

# Rows in I3MCWeightDict that are missing from HESE_CausalQTot
missing = merged[merged["_merge"] == "left_only"]

print(f"Number of missing events: {len(missing)}")
# print(missing[missing["Event"] < 100])

# we have a problem in creating the hdf file of 
# /data/sim/IceCube/2020/filtered/level3/cascade/neutrino-generator/22043/0000000-0000999/Level3_NuMu_NuGenCCNC.022043.000196.i3.zst
# hdf_test = pd.HDFStore("/data/sim/IceCube/2020/filtered/level3/cascade/neutrino-generator/22043/0000000-0000999/Level3_NuMu_NuGenCCNC.022043.000196.i3.zst",'r')



88234 88234 83872 83872
Number of missing events: 4362


In [ ]:
data = {}

key = "spice_l3casc"

simulation_dataset = simulation_datasets[key]

channel_data = {}
channel_data_masked = {}

for flavor in ["NuMu_highE1","NuMu_midE2", "NuMu_highE2"]: # "NuMu_midE1" was/is corrupt
    weights = simulation_dataset[flavor]["weighter"].get_weights(AstroFluxModel) * livetime_s
    rate = np.sum(weights)
    error = np.sqrt(np.sum(weights**2))
    channel_data[f"astro_{flavor}"] = f"{rate:.2f} ± {error:.2f}"

    HESE_CausalQTot = simulation_dataset[flavor]["hdf_file"]["HESE_CausalQTot"].value
    mask = HESE_CausalQTot > 6000
    rate_masked = np.sum(weights[mask])
    error_masked = np.sqrt(np.sum(weights[mask]**2))
    channel_data_masked[f"astro_{flavor}"] = f"{rate_masked:.2f} ± {error_masked:.2f}"

data[key] = channel_data
data[f"{key}_masked"] = channel_data_masked

# Create DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Display as string table
print(df.to_string())

                    astro_NuMu_highE1 astro_NuMu_midE2 astro_NuMu_highE2
spice_l3casc              9.31 ± 0.13    919.78 ± 2.97       9.19 ± 0.08
spice_l3casc_masked       0.81 ± 0.04     13.77 ± 0.23       0.82 ± 0.02


Wow!! It turns out, if I do the cut myself on spice, I get the same number. How did Neha get higher values? Probably due to her definitions in 
https://github.com/icecube/wg-diffuse/blob/2023_GlobalFit_Flavor/Ternary_Classifier/segments/VHESelfVeto.py